# The Sentinel-2 class

*Author: Gregor Perich*

In this notebook, we will take a closer look at the `Sentinel2` [class](https://github.com/EOA-team/eodal/blob/master/eodal/core/sensors/sentinel2.py) of EOdal. The `Sentinel2` class has many methods and attributes specifically tailored to the data structure of the [Sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) satellites of the European Space Agency (ESA). Therefore, we begin with a quick introduction of the Sentinel-2 (S2) satellites. 

## The Sentinel-2 satellites

S2 is a pair of satellites (Sentinel-2A and 2B) in sun-synchronous orbit of ca. 800 km. The satellites carry the MultiSpectral Instrument ([MSI](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi)) which features 13 bands ranging from 10 to 60 m spatial resolution. The spectral resolution ranges from the VIS to the SWIR range.

*Spectral and spatial resolution of the Sentinel-2 bands. Figure from ESA.*

<img src="../images_for_text/S2_spectral_resolution.png" width="500"/>


The S2 satellites have an individual revisit frequency of ca. 10 days, combined they achive ca. 5 days. The revisit frequency is highly dependent on the overlap of the orbits. This leads to certain regions (especially in the mid latitudes ~45°N/S) having considerably fewer observations than others. Please see the image below for an illustration or read more on the [ESA webpage](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/revisit-coverage).

<img src="../images_for_text/S2_orbit_overlap.jpg" width="750"/>

*Geometric revisit frequency due to the individual S2 orbits. Figure from ESA*


Each S2 satellite has an orbital swath width of 290 km. This means that with every orbit, a 290 km strip is imaged by the MSI. This data strip is processed into individual 100 x 100 km [tiles](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/overview). The figure below illustrates this. Often, a user gets these tiles as data to work with, which gets especially tricky in the border regions and at the start/finish of an individual S2 data strip. Luckily, EOdal takes care of these operations. 

<img src="../images_for_text/S2_from_swath_to_tile.png" width="500"/>

*Illustration of how the S2 tiles are being generated from the original data strip. Figure from ESA*

You can get S2 data in two processing levels from ESA: [Level 1C](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/processing-levels/level-1) (L1C) and [Level 2A](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/processing-levels/level-2) (L2A). L1C is a top-of-atmosphere data product, L2A is bottom-of-atmosphere. Unless you want to do the atmospheric correction yourself, I recommend you work with the L2A level. In our data query below, we need to filter for the processing level specifically. 

### The Scene Classification Layer - SCL
The Scene Classification Layer - SCL comes part of the bottom-of-atmosphere [L2A product](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm-overview) provided by ESA. The SCL layer contains 12 classes (indexed from 0 to 11) and can be readily used as a cloud masking algorithm, or even a high-level land use classifier. 

<img src="../images_for_text/SCL_classes.png" width="300">

*SCL layers provided with the L2A product of Sentinel-2. Figure from ESA*

## Get S2 data

Here, we 

**TODO: Run the whole process of getting data for visualisation**

- define AOI
- define SCL mask (explain SCL)
- cloud filtration based on SCL



In [6]:
# import libraries and set paths

import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
from eodal.config import get_settings
from eodal.core.sensors.sentinel2 import Sentinel2  # native support for Sentinel2
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs
from typing import List
from pathlib import Path

base_dir: Path = Path('../')
data_dir: Path = Path(base_dir.joinpath("data"))
temp_dir: Path = Path(base_dir.joinpath("temp_files"))

# Define settings, s.t. we can access the data from MS planetary computer
Settings = get_settings()
Settings.USE_STAC = True

`eodal` has built-in support for S2 data as you can see from the imports above. Now, let's define our helper functio to save the S2 data in the `Sentinel2` class. Since we are using the `Sentinel2` [class](https://github.com/EOA-team/eodal/blob/master/eodal/core/sensors/sentinel2.py), we can use the class-specific methods and attributes, such as the `mask_clouds_and_shadows` method. This method uses certain SCL classes as default, but you can supply your own. 

The `resample` method is part of the `band` [class](https://github.com/EOA-team/eodal/blob/master/eodal/core/band.py) of `eodal`. 

In [7]:
def preprocess_sentinel2_scenes(
    ds: Sentinel2,
    target_resolution: int,
) -> Sentinel2:
    """
    Resample Sentinel-2 scenes and mask clouds, shadows, and snow
    based on the Scene Classification Layer (SCL).

    :param target_resolution:
        spatial target resolution to resample all bands to.
    :returns:
        resampled, cloud-masked Sentinel-2 scene.
    """
    
    # resample scene
    # default interpolation method is cv2.INTER_NEAREST_EXACT
    # but you can specify other methods from cv2 or your own
    ds.resample(inplace=True, target_resolution=target_resolution)
    
    # mask clouds, shadows, and snow
    # the defaults are SCL classes [1, 2, 3, 7, 8, 9, 10, 11]
    # but you can also specify your own list of classes
    ds.mask_clouds_and_shadows(inplace=True)
    
    return ds

### User Input
Here, we have to define our query to get S2 data: E.g. define the date range, the AOI, the type of data (L1C vs L2A), which bands to select, etc. 

The `from_safe` [method](https://github.com/EOA-team/eodal/blob/52d456fbd2e1485db6808df54280008cfa4c2cef/eodal/core/sensors/sentinel2.py#LL232C13-L232C13) called below tells `eodal` to read the data directly according to ESA's .SAFE [data structure](https://sentinel.esa.int/documents/247904/685211/Sentinel-2-Products-Specification-Document.pdf/fb1fc4dc-12ca-4674-8f78-b06efa871ab9).

In [11]:
# -------------------------- Collection -------------------------------
collection: str = "sentinel2-msi"

# ------------------------- Time Range ---------------------------------
time_start: datetime = datetime(2023, 5, 12)  # year, month, day (incl.)
time_end: datetime = datetime(2023, 6, 12)  # year, month, day (incl.)

# ---------------------- Spatial Feature  ------------------------------
geom: Path = Path(data_dir.joinpath("eschikon_bbox.gpkg"))

# ------------------------- Metadata Filters ---------------------------
metadata_filters: List[Filter] = [
    Filter("cloudy_pixel_percentage", "<", 25),
    Filter("processing_level", "==", "Level-2A"),
]

# -------- Define kwargs for the scene constructor ---------------------
scene_kwargs = {
    "scene_constructor": Sentinel2.from_safe,
    "scene_constructor_kwargs": {"band_selection": None}, # defaults to all 10 and 20 m bands
    "scene_modifier": preprocess_sentinel2_scenes,  # our function from above
    "scene_modifier_kwargs": {"target_resolution": 10},
}

# query the scenes available (no I/O of scenes, this only fetches metadata)
feature = Feature.from_geoseries(gpd.read_file(geom).geometry)
mapper_configs = MapperConfigs(
    collection=collection,
    time_start=time_start,
    time_end=time_end,
    feature=feature,
    metadata_filters=metadata_filters,
)

# create a Mapper instance
mapper = Mapper(mapper_configs)

# fetch metadata
mapper.query_scenes()

Now, we can take a look at the available scenes by looking at the metadata of our query. From the `product_uri` column, we see that we fetched the desired L2A data correctly (MSIL2A). We can also see additional important info, such as the S2 [tile](https://eatlas.org.au/data/uuid/f7468d15-12be-4e3f-a246-b2882a324f59) and if said tile has been mosaicked internally by `eodal` or not (mosaicing column).

In [12]:
mapper.metadata

,product_uri,scene_id,spacecraft_name,tile_id,sensing_date,cloudy_pixel_percentage,epsg,sensing_time,sun_azimuth_angle,sun_zenith_angle,geom,assets
5,S2A_MSIL2A_20230526T101601_R065_T32TMT_2023052...,S2A_OPER_MSI_L2A_TL_MSFT_20230526T173431_A0413...,Sentinel-2A,32TMT,2023-05-26,21.977860,32632,2023-05-26 10:16:01.024,152.054531,28.539268,"POLYGON ((7.74149 46.85845, 7.77781 46.96052, ...",{'AOT': {'href': 'https://sentinel2l2a01.blob....
4,S2A_MSIL2A_20230529T102601_R108_T32TMT_2023052...,S2A_OPER_MSI_L2A_TL_MSFT_20230529T194846_A0414...,Sentinel-2A,32TMT,2023-05-29,1.837260,32632,2023-05-29 10:26:01.024,156.249533,27.361099,"POLYGON ((7.66287 47.84591, 9.13047 47.85363, ...",{'AOT': {'href': 'https://sentinel2l2a01.blob....
3,S2B_MSIL2A_20230531T101559_R065_T32TMT_2023053...,S2B_OPER_MSI_L2A_TL_MSFT_20230531T152326_A0325...,Sentinel-2B,32TMT,2023-05-31,3.746394,32632,2023-05-31 10:15:59.024,151.185621,27.848905,"POLYGON ((7.74644 46.85848, 7.76865 46.92095, ...",{'AOT': {'href': 'https://sentinel2l2a01.blob....
2,S2B_MSIL2A_20230603T102559_R108_T32TMT_2023060...,S2B_OPER_MSI_L2A_TL_MSFT_20230603T155551_A0325...,Sentinel-2B,32TMT,2023-06-03,7.403748,32632,2023-06-03 10:25:59.025,155.435580,26.748966,"POLYGON ((7.66287 47.84591, 9.13047 47.85363, ...",{'AOT': {'href': 'https://sentinel2l2a01.blob....
1,S2A_MSIL2A_20230605T101601_R065_T32TMT_2023060...,S2A_OPER_MSI_L2A_TL_MSFT_20230605T183124_A0415...,Sentinel-2A,32TMT,2023-06-05,23.888430,32632,2023-06-05 10:16:01.024,150.323335,27.326367,"POLYGON ((7.74306 46.85846, 7.76160 46.91137, ...",{'AOT': {'href': 'https://sentinel2l2a01.blob....
0,S2A_MSIL2A_20230608T102601_R108_T32TMT_2023060...,S2A_OPER_MSI_L2A_TL_MSFT_20230608T195154_A0415...,Sentinel-2A,32TMT,2023-06-08,22.512731,32632,2023-06-08 10:26:01.024,154.626261,26.306467,"POLYGON ((7.66287 47.84591, 9.13047 47.85363, ...",{'AOT': {'href': 'https://sentinel2l2a01.blob....


In [13]:
# get actual data, this is the I/O step
mapper.load_scenes(scene_kwargs=scene_kwargs)

# store the data (type = SceneCollection) separately
scene_coll = mapper.data

2023-06-13 17:26:37,180 eodal        INFO     Starting extraction of sentinel2 scenes
2023-06-13 17:27:06,340 eodal        INFO     Finished extraction of sentinel2 scenes


The individual scenes of our `SceneCollection` are stored as `RasterCollections` of the `Sentinel2` class. We have all S2 bands (except the 60 m ones) and the SCL layer. 

In [27]:
# select a single scene using the timestamps of the SceneCollection
scene = scene_coll[scene_coll.timestamps[0]]
print(f'The scene is of type {type(scene)}')

scene

The scene is of type <class 'eodal.core.sensors.sentinel2.Sentinel2'>


EOdal RasterCollection
----------------------
# Bands:    11
Band names:    B02, B03, B04, B05, B06, B07, B08, B8A, B11, B12, SCL
Band aliases:    blue, green, red, red_edge_1, red_edge_2, red_edge_3, nir_1, nir_2, swir_1, swir_2, scl

*TODO: Get an AOI in western CH and one in eastern CH to show the difference of the orbits*